Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

## Running Inceptionv3 as an Azure Machine Learning Service 

This notebook guides you to register Inception-v3 model in Azure Machine Learning Service, deploy it as a web service, and run inference by providing input images.  

### Download Inception V3 Models

In [ ]:
import time

start = time.time()

!wget -nc -P model/ http://download.tensorflow.org/models/image/imagenet/inception-2015-12-05.tgz
!tar xvzf model/inception-2015-12-05.tgz -C model/
!rm model/cropped_panda.jpg
!rm model/inception-2015-12-05.tgz
    
end = time.time()
print("Model download latency: {} seconds".format(end-start))

### Load AML Workspace

In [ ]:
from azureml.core import Workspace
ws = Workspace.from_config()
print(ws.name, ws.subscription_id, ws.resource_group, ws.location)

### Register Inception-v3 Models in Azure Machine Learning Service

You can use following python code snippets to register models to Model Management Service provided by Azure Machine Learning.

In [ ]:
from azureml.core.model import Model

models = []

start = time.time()
new_model = Model.register(
    workspace = ws,
    model_path = "model/classify_image_graph_def.pb",
    model_name = "classify_image_graph_def")
models.append(new_model)

end = time.time()
print("Register model latency: {} seconds".format(end-start))
print("Model id={}, name={}, created time={}".format(
    new_model.id, 
    new_model.name, 
    new_model.created_time))

In [ ]:
start = time.time()
new_model = Model.register(
    workspace = ws,
    model_path = "model/imagenet_2012_challenge_label_map_proto.pbtxt",
    model_name = "imagenet_label_map")
models.append(new_model)

end = time.time()
print("Register model latency: {} seconds".format(end-start))
print("Model id={}, name={}, created time={}".format(
    new_model.id, 
    new_model.name, 
    new_model.created_time))

In [ ]:
start = time.time()
new_model = Model.register(
    workspace = ws,
    model_path = "model/imagenet_synset_to_human_label_map.txt",
    model_name = "imagenet_s2h_label_map")
models.append(new_model)

end = time.time()
print("Register model latency: {} seconds".format(end-start))
print("Model id={}, name={}, created time={}".format(
    new_model.id, 
    new_model.name, 
    new_model.created_time))

### Build Container Image 

Next, we will create a docker image which runs a flask web service. The web service has multiple endpoints to serve the inception-v3 models.

In [ ]:
from azureml.core.image import Image, ContainerImage

image_config = ContainerImage.image_configuration(
    execution_script = "score.py",
    runtime = "python", 
    dependencies = ["app.py", "ui.html", "inception.py"],
    conda_file = "env.yml")

start = time.time()
image = Image.create(
    workspace = ws,
    name = "inception-v3-demo-image",
    models = models,
    image_config = image_config
)

image.wait_for_creation(show_output = True)

end = time.time()
print("Image creation latency: {} seconds".format(end-start))

### Deploy Image to ACI

After we successfully create the image, we will use Azure Machine Leaning's python SDK to deploy it via Azure Container Instances (ACI)

In [ ]:
from azureml.core.webservice import AciWebservice, Webservice

aci_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)

start = time.time()
aci_service = Webservice.deploy_from_image(
    workspace = ws,
    name = "inception-v3-demo-service",
    image = image,
    deployment_config = aci_config)
aci_service.wait_for_deployment(show_output = True)

end = time.time()
print("Service deployment latency: {} seconds".format(end-start))

if (aci_service.state != 'Healthy'):
    raise ValueError("service deployment is in bad state: {}".format(aci_service.state))

### Run Inferences

There are a few different ways to call the web service we just deployed to classify images. 

**Run Inception-v3 interactively**

The web service provides a UI page. You can drag and drop image and receive the prediction in real-time

In [ ]:
# Get interactive demo URL 
scoring_uri = aci_service.scoring_uri
demo_uri = scoring_uri.replace("/score", "/ui")
print("Try out the inception-v3 model at {}".format(demo_uri))

**Run Inception-v3 programmatically**

The web serivce also provides a POST REST API which you can call by providing the binary data of an image. Below is an example of calling that REST API with one of the sample images included in this repository. Try that images as well :). 

In [ ]:
from IPython.display import Image
imageFilename = "SampleImages/dog.jpg"
dogImage = Image(imageFilename)
print("Here is the image:")
dogImage

In [ ]:
print("Here is the prediction of Inception-v3 model:")
!curl $scoring_uri --data-binary @$imageFilename -H "Content-Type: application/octet-stream"

### Cleanup Resources

Delete the web serivce, image, and models

In [ ]:
aci_service.delete()
image.delete()
for m in models:
    m.delete()
print("Successfully deleted models, image and service.")